- <span style='color:red'> Change number of candidates (x) in ... 
    - Function call (rerank_suggs_by_prob) and
    - Pickle export ("...\_**x**c.pkl")</span>



1. Read file
    - generated chunks results
    - Prepare dataframe for genearting new suggestion list
2. RERANK

3. Merge to original df
    - Get old dataframe
    - Merge with new suggestion list
4. Evaluate

In [1]:
import pandas as pd

# Do not truncate columns of DataFrame
pd.set_option('display.max_rows', None)
# Do not truncate column width
pd.set_option('display.max_colwidth', None)

import helpers_reranking as helpers

## 1 - Read file

In [2]:
#df = pd.read_csv('chunk_generation_2021-12-08_results.csv', sep='|')
# TODO: path + filename
#path = r'./Output_chunk/chunk_generation_2022-01-04_boyd_results.csv'
path = r'./Output_chunk/chunk_generation_2022-01-12_boyd_results.csv'
df = pd.read_csv(path, sep='|')

In [3]:
df.tail(10)

,chunk,candidate,err_idx,num_chunk_build,prob
246000,Lea den Lutscher zu schenken,Lutscher,210760,24600,-21.890709
246001,Lea den lutsche zu schenken,lutsche,210760,24600,-114.633781
246002,Lea den Lutschen zu schenken,Lutschen,210760,24600,-21.714617
246003,Lea den lutschten zu schenken,lutschten,210760,24600,-114.633781
246004,Lea den Lutscht zu schenken,Lutscht,210760,24600,-114.633781
246005,Lea den lutschte zu schenken,lutschte,210760,24600,-22.015646
246006,Lea den lutschen zu schenken,lutschen,210760,24600,-21.714617
246007,Lea den Lutschphasen zu schenken,Lutschphasen,210760,24600,-114.633781
246008,Lea den lutscht zu schenken,lutscht,210760,24600,-114.633781
246009,Lea den Löscher zu schenken,Löscher,210760,24600,-22.492767


In [4]:
df.head(40)
len(df)

246010

## 2 - Rerank

In [5]:
# Insert loop here
test_df = helpers.rerank_suggs_by_prob(df, set_candidates=2)

In [6]:
test_df.head(10)
new_df = test_df

## 3 - Merge to original dataframe
- import old df
- prepare
- merge
- clean df from unnecessary columns; Goal: Look like normal analysis df, be able to evaluate

In [7]:
#old_df=pd.read_pickle('data_error_token_language_tool_whole_texts_evaluation_lc_ALL_COLUMNS_OFFSETS_OTHERINDEX_20211208_TOMERGEWITH.pkl')
old_df=pd.read_pickle('data_error_types_boyd_evaluation_cs_preprocessed_reranking.pkl')

In [8]:
#old_df.tail(50)

In [9]:
# Change
old_df.rename(columns={'index':'err_idx'}, inplace=True)

In [10]:
old_df.head(2)
#old_df.shape[0]

,err_idx,original,corrected,filename,num_texts,length,start_position,end_position,ct,freq_ori,freq_cor,freq_tup,idx_df_orig,suggestions
0,10,belt,bellt,01-005-2-III-Eis.csv,0,4,46,50,10,91,138,91,0,"[ballt, bellt, beult, bellst, hebelt, beeilt, pellt, ballte, Bild, gelt]"
1,17,kukt,kuckt,01-005-2-III-Eis.csv,0,4,80,84,17,73,152,73,1,"[Guckt, kuckt, kickt, guckt, Guckst, guckst, kickst, Guckte, kickte, kickten]"


In [11]:
merge_df=old_df.merge(new_df, how='left', on='err_idx')

In [12]:
merge_df.shape[0] # 24601
merge_df.head(2)

,err_idx,original,corrected,filename,num_texts,length,start_position,end_position,ct,freq_ori,freq_cor,freq_tup,idx_df_orig,suggestions,num_chunk_build,new_suggestions
0,10,belt,bellt,01-005-2-III-Eis.csv,0,4,46,50,10,91,138,91,0,"[ballt, bellt, beult, bellst, hebelt, beeilt, pellt, ballte, Bild, gelt]",0,"[bellt, ballt]"
1,17,kukt,kuckt,01-005-2-III-Eis.csv,0,4,80,84,17,73,152,73,1,"[Guckt, kuckt, kickt, guckt, Guckst, guckst, kickst, Guckte, kickte, kickten]",1,"[Guckt, kuckt]"


In [13]:
# Do not truncate df value
pd.set_option('display.max_colwidth', None)

In [14]:
# Clean df from columns
#gr = ['err_idx','num_texts','length','start_position','end_position','ct','rule_id','in_sugg','sugg_idx','idx0','idx_df_orig','num_chunk_build']
gr = ['err_idx','num_texts','length','start_position','end_position','ct','idx_df_orig','num_chunk_build']
merge_df.drop(axis=1,inplace=True,labels=gr)

In [15]:
merge_df.tail(4)

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions,new_suggestions
24597,ass,aß,10-693-4-IV-Weg.csv,3,7,3,"[Ass, aß, nass, pass, lass, bass, fass, dass, aus, hasst]","[aß, Ass]"
24598,mutzte,nutzte,10-693-4-IV-Weg.csv,1,1,1,"[nutzte, motzte, benutzte, muckste, nützte, nutzten, putzte, benützte, benutzten, benutztes]","[nutzte, motzte]"
24599,momen,Moment,10-693-4-IV-Weg.csv,1,4,1,"[Moment, Nomen, Moments, mopsen, Momenten, Momente, Normen, benommen, Nomens, Mom]","[Moment, Nomen]"
24600,Lutscha,Lutscher,10-693-4-IV-Weg.csv,8,56,8,"[Lutscher, lutsche, Lutschen, lutschten, Lutscht, lutschte, lutschen, Lutschphasen, lutscht, Löscher]","[Lutscher, lutsche]"


In [16]:
#merge_df.tail(100)

In [17]:
merge_df_new = merge_df.drop(axis=1,labels='suggestions')

In [18]:
merge_df_new.rename(columns={'new_suggestions':'suggestions'}, inplace=True)

In [19]:
merge_df_new.head(20)

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,"[bellt, ballt]"
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,"[Guckt, kuckt]"
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,"[dann, Dan]"
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,"[gekriegt, gekracht]"
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,"[lenken, lecken]"
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,"[fehlt, fallt]"
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,"[wollte, Wollte]"
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,"[legt, leckt]"
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,"[vom, Vom]"
9,dan,dann,01-005-2-III-Eis.csv,627,651,621,"[dann, Dan]"


## 4 - Evaluate

In [20]:
# EVALUIEREN
# Prep
merge_df_new.reset_index(drop=True, inplace=True)
# Not needed because there are suggestions for every type
#merge_df_new.suggestions.fillna(value='',inplace=True)

In [21]:
import sys
sys.path.insert(0, '../../Lisa')
import litkey_2
import helpers_litkey_2 as helpers

In [23]:
token_eval_lc = litkey_2.evaluate(merge_df_new)

  0%|          | 0/24601 [00:00<?, ?it/s]

In [24]:
token_eval_cs = litkey_2.evaluate(merge_df_new, lower_case=False)

  0%|          | 0/24601 [00:00<?, ?it/s]

In [26]:
#token_eval_cs.head(50)

In [ ]:
# To pickle
#token_eval_lc.to_pickle('./pickles/reranking_data_error_token_language_tool_whole_texts_evaluation_lc.pkl')
#token_eval_cs.to_pickle('./pickles/reranking_data_error_token_language_tool_whole_texts_evaluation_cs.pkl')

In [27]:
# TODO: Set path dynamically
#token_eval_lc.to_pickle('./pickles/reranking_data_error_token_boyd_evaluation_lc_3c.pkl')
#token_eval_cs.to_pickle('./pickles/reranking_data_error_token_boyd_evaluation_cs_3c.pkl')

#token_eval_lc.to_pickle('./pickles/reranking_data_error_token_boyd_evaluation_lc_5c.pkl')
#token_eval_cs.to_pickle('./pickles/reranking_data_error_token_boyd_evaluation_cs_5c.pkl')

token_eval_lc.to_pickle('./pickles/reranking_data_error_token_boyd_evaluation_lc_2c.pkl')
token_eval_cs.to_pickle('./pickles/reranking_data_error_token_boyd_evaluation_cs_2c.pkl')

In [ ]:
#token_eval_lc = pd.read_pickle('./pickles/reranking_data_error_token_boyd_evaluation_lc.pkl')
#token_eval_cs = pd.read_pickle('./pickles/reranking_data_error_token_boyd_evaluation_cs.pkl')

In [ ]:
#token_eval_cs.head(5)